# AMLD workshop by L2F – Learn to Forecast

Welcome to the "Machine Learning Competition: Tennis Analysis" workshop at AMLD 2019, organized by [L2F – Learn to Forecast](https://www.l2f.ch/)!

Starting from Jeff Sackman's crowdsourced [Match Charting Project](https://github.com/JeffSackmann/tennis_MatchChartingProject) dataset, we have put together for you a dataset containing detailed shot-by-shot descriptions of a large collection of points played between the Swiss tennis star Roger Federer and his historical rival Rafael Nadal (RN). The information recorded includes the type of shot, direction of shot, depth of returns, types of errors, and more. This is better explained below; we have also put together some preliminary data preparation to help you get started.

## Goal

The goal of this workshop is for you to come up with an optimized Phederos – the god of tennis – who will be able to destroy his ancient human rival. Do you think you can beat the beast?

From a more technical perspective, we ask you to provide a strategy function which, given an opponent's shot, will return what you believe to be an optimal response shot. Your answer must be in the form of a JSON dictionary as follows: each key is an opponent's shot, and the corresponding value is a list from which we will uniformly sample your response.

For more clarity, let us consider an example of such a strategy (the strings denote shot types which are better explained below):

*strat* = {'f3': ['b2', 'b2', 'b3', 'v1'], 'f1': ['f2'], ...}.

The keys in this dictionary are shots RN can produce. Suppose RN is performing a forehand on your backhand (the code for this shot is 'f3'); *strat* would reply to this shot in a *probabilistic* way as follows: 
- 'b2' with probability 1/2,
- 'b3' with probability 1/4,
- 'v1' with probability 1/4.

On the other hand, the same strategy replies *deterministically* to 'f1' (which is a forehand on our forehand), since we say 'f2' (a forehand straight to RN) with probability 1. In this way, your strategy can incorporate simple deterministic and more complex probabilistic decision-making in an arbitrary way.

## Testing environment

You have to beat a stochastic model which replies to your shots based on past statistics, but never replies with shots which have historically occurred too rarely. More precisely, this simulation environment will react based on the counting statistics of outcomes to (human) Federer's shots. By 'outcome' here we mean either:
- a simple reply shot or serve by historical RN (this is perhaps a bit confusing at first, but we consider a player's serve as a 'reply' to the opponent's 'waiting');
- a 'termination token', e.g. '*' for a winning shot, meaning that RN could not respond to your winning shot and the point terminates, or '@' indicating that with your shot you made an unforced error and lost the point – but note that such tokens can include more detailed information than this and be more than one character long;
- a shot by RN and a termination token, concatenated in a simple string, e.g., 'f1*' would indicate that our simulated RN replied with a winning forehand shot – ouch!

The Python code for this tester can be found in the last part of this notebook.

### Important remarks

- The simulation environment will prompt you with shots or termination/waiting tokens from a predetermined list, which can be accessed by loading the pickle file *possible_prompts.p*. Make sure you know how to react to each of the elements in this list!
- The pickle file *allowed_shots.p* contains a list which can be obtained from the list in *possible_prompts.p* by removing termination and waiting tokens. Replying with a string not in this list leads to losing of the point. 
- The "past statistics" in the stochastic model you will be fighting against come from a hold-out "test set" and **not** from the dataset you have access to. Make sure to build models which generalize well!
- We will impose a time constraint on the duration of the evalutation of your policy: we expect that your policy can be evaluated on an i7 quadcore 1000000 times in less than 3 minutes.

Will the true heroes of Machine Learning rise? :)

# Data from Jeff Sackmann's GitHub

As explained above, we provide you with a pandas dataframe (*dataframe.p*), taken from Jeff Sackmann's original dataset. 

Each row is one point:
- the variables 'Player1' and 'Player2' are the name of the players
- the 'match_id' variable identifies the tournament and date
- the varaible 'Shots' is a string containing all the shots, played alternatively from the players, during the point
- the variable 'Shots1' are the shots played only by RF
- the variable 'Shots0' contain the shots played only by RN
- The 'Outcome' variable describes the outcome of the point
- The 'Server' variable has boolean values: it is 1 in case RF is serving
- The 'Winner' variable has boolean values: it is 1 in case RF has won the point

Numbers are used to indicate direction and depth, while letters are used to specific shot types (e.g. 'f' stands for 'forehand') and error types ('n' stands for 'net'). The letter 'X' means that RN is waiting for RF to serve... don't let him wait too much! A few symbols are used for other purposes, such as types of errors (e.g. '@' means 'unforced error' and '+' indicates an approach shot).

To understand the codes for each shot, we prepared an explanation file named "Shot encodings" with more details. You can also check [this example video](https://drive.google.com/open?id=1Q4obFagfzxkFy_mq9C5gWITQiXxBQfCf) for a visual demonstration of this way of encoding.

In [1]:
# Importing libraries

import numpy as np
import pickle
import pandas as pd
from nltk import ngrams

In [2]:
# Historical data

df = pd.read_pickle('dataframe.p')
df.head()

,match_id,Player1,Player2,Shots,Shots1,Shots0,Outcome,Server,Winner
0,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b;37 f1 f1,b;37 f1,4 f1,*,0,1
1,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,6 b3,b3,6,w#,0,0
2,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,6,,6,*,0,0
3,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,5 b18 b2 f1 b2 f1 s2 b+3,b18 f1 f1 b+3,5 b2 b2 s2,*,0,1
4,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b27 f+3 b1,b27 b1,4 f+3,*,0,1


In [3]:
server_nadal = df[df.Server == 0]
server_nadal_lost = server_nadal[server_nadal.Winner == 1]
len(server_nadal), len(server_nadal_lost)

(2256, 809)

In [4]:
server_nadal.head()

,match_id,Player1,Player2,Shots,Shots1,Shots0,Outcome,Server,Winner
0,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b;37 f1 f1,b;37 f1,4 f1,*,0,1
1,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,6 b3,b3,6,w#,0,0
2,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,6,,6,*,0,0
3,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,5 b18 b2 f1 b2 f1 s2 b+3,b18 f1 f1 b+3,5 b2 b2 s2,*,0,1
4,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b27 f+3 b1,b27 b1,4 f+3,*,0,1


In [5]:
server_nadal_lost.head()

,match_id,Player1,Player2,Shots,Shots1,Shots0,Outcome,Server,Winner
0,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b;37 f1 f1,b;37 f1,4 f1,*,0,1
3,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,5 b18 b2 f1 b2 f1 s2 b+3,b18 f1 f1 b+3,5 b2 b2 s2,*,0,1
4,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b27 f+3 b1,b27 b1,4 f+3,*,0,1
12,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,5 b28 f3 b2 b;2 f1 b1 f1,b28 b2 f1 f1,5 f3 b;2 b1,*,0,1
19,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 f17 b1 f3 f2 f1 b2,f17 f3 f1,4 b1 f2 b2,d@,0,1


In [49]:
nadal_serve_win_response = {}
nadal_serve_lost_response = {}


for index, row in server_nadal.iterrows():
    shots_s = row.Shots.split(' ')
    for k in range(0, len(shots_s) - 1, 2):
        n_shot, f_shot = shots_s[k:k+2]
        # Federer wins
        if row.Winner == 1:
            if n_shot in nadal_serve_win_response:
                nadal_serve_win_response[n_shot].append(f_shot)
            else:
                nadal_serve_win_response[n_shot] = [f_shot]
        # Nadal wins
        if row.Winner == 0:
            if n_shot in nadal_serve_lost_response:
                nadal_serve_lost_response[n_shot].append(f_shot)
            else:
                nadal_serve_lost_response[n_shot] = [f_shot]

In [50]:
server_federer = df[df.Server == 1]

In [51]:
nadal_serve_win_response

{'4': ['b;37',
  'b27',
  'f17',
  'f18',
  'b28',
  'f38',
  'f28',
  'b18',
  'f29',
  'f18',
  'b9',
  'b37',
  'f18',
  'b8',
  's18',
  's39',
  'b19',
  'b28',
  'f29',
  'b37',
  's28',
  'b19',
  's29',
  'b28',
  'b28',
  'b28',
  'f29',
  'b28',
  'b2',
  'b3',
  'b1',
  'b2',
  'b2',
  'b2',
  'b28',
  'f18',
  'b29',
  'b38',
  's38',
  'b37',
  'b37',
  'f18',
  'f19',
  'b38',
  'b18',
  'f19',
  'b28',
  'b27',
  'b28',
  'f17',
  'b18',
  'f19',
  'b28',
  'b28',
  'b39',
  'b29',
  'f38',
  'f18',
  'f17',
  'b18',
  'b19',
  'b27',
  'b38',
  'b28',
  'b19',
  'b18',
  'f18',
  'b29',
  'b27',
  'b19',
  'b18',
  'b38',
  'b27',
  'f18',
  'b18',
  'b18',
  'b38',
  'b;27',
  'b;27',
  's38',
  'b29',
  'b19',
  'f39',
  's38',
  's28',
  's28',
  's37',
  's28',
  'f28',
  's29',
  'b37',
  'f28',
  'b28',
  's37',
  's38',
  'b38',
  's37',
  's28',
  'b28',
  'b18',
  's37',
  'b28',
  'b37',
  'b28',
  'b27',
  'b28',
  'b27',
  's28',
  'b28',
  'b28',
  'b17',
 

In [52]:
server_federer_won = server_federer[server_federer.Winner == 1]
server_federer_lost = server_federer[server_federer.Winner == 0]

In [53]:
server_federer

,match_id,Player1,Player2,Shots,Shots1,Shots0,Outcome,Server,Winner
5,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b2,4,b2,d#,1,1
6,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 f37 s1,4 s1,f37,n#,1,0
7,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b28 b2 s3 f+1 b1,4 b2 f+1,b28 s3 b1,w#,1,1
8,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4,4,,*,1,1
9,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,5 f37 s1 b1,5 s1,f37 b1,w@,1,1
13,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 s38 f1 b1 f1 b+3 s3,4 f1 f1 s3,s38 b1 b+3,n#,1,0
14,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,6 b27 f+3 f3 z1 m3 o3,6 f+3 z1 o3,b27 f3 m3,*,1,1
15,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b29 f1 s2 f3 f3 b2 f2 b2 b2 f1 b2 b2 b1 f1,4 f1 f3 b2 b2 f1 b2 f1,b29 s2 f3 f2 b2 b2 b1,*,1,1
16,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 f238 f2 f3 b3 f3,4 f2 b3,f238 f3 f3,w@,1,1
17,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b28 f3,4 f3,b28,d@,1,0


In [56]:
#federer_serve_win_response = {}
#for i in server_federer_won.Shots.values:
#    shots_s = i.split(' ')
#    if len(shots_s) < 3:
#        continue
#    for k in range(1, len(shots_s) - 1, 2):
#        n_shot, f_shot = shots_s[k:k+2]
#        if n_shot in federer_serve_win_response:
#            federer_serve_win_response[n_shot].append(f_shot)
#        else:
#            federer_serve_win_response[n_shot] = [f_shot]
            
            
federer_serve_win_response = {}
federer_serve_lost_response = {}

for index, row in server_federer.iterrows():
    shots_s = row.Shots.split(' ')
    for k in range(1, len(shots_s) - 1, 2):
        n_shot, f_shot = shots_s[k:k+2]
        # Federer wins
        if row.Winner == 1:
            if n_shot in federer_serve_win_response:
                federer_serve_win_response[n_shot].append(f_shot)
            else:
                federer_serve_win_response[n_shot] = [f_shot]
        # Nadal wins
        if row.Winner == 0:
            if n_shot in federer_serve_lost_response:
                federer_serve_lost_response[n_shot].append(f_shot)
            else:
                federer_serve_lost_response[n_shot] = [f_shot]

In [59]:
promp_dict = ['']

In [58]:
federer_serve_lost_response

{'f37': ['s1',
  's1',
  's3',
  'f3',
  'f1',
  'b1',
  's+3',
  's3',
  'b1',
  's+1',
  'b+2',
  'b2',
  'b1',
  'b3',
  'f3',
  'b3',
  'b3',
  'b3',
  's+2',
  'z1',
  'b3',
  'f2',
  'b1',
  'u+3',
  'b3',
  'f1',
  'f+2',
  'b+1',
  's1',
  'b3',
  'b+1',
  's1',
  'f+3',
  'f3',
  'b3',
  'f2',
  'f3',
  'f1',
  'y3',
  'b2',
  's2',
  'f3',
  'b+1',
  'f+3',
  'f1',
  'b+1',
  'b3',
  'f3',
  'b1',
  'f3',
  'f3',
  'b2',
  'f+1',
  'b3',
  's1',
  'b1',
  'b1',
  'f3',
  'f3',
  'f3',
  'f+2',
  's2'],
 's38': ['f1', 'f1', 'f3', 'f1', 'b1', 'b1', 'f+1'],
 'b1': ['f1',
  'f2',
  'r1',
  'f2',
  'f1',
  'r',
  'f1',
  'f1',
  'r2',
  'f3',
  'f1',
  'f3',
  'u3',
  'f2',
  'f2',
  'v3',
  'f3',
  'f1',
  'f1',
  'f1',
  'f3',
  'v3',
  'f3',
  'f1',
  'f2',
  'f3',
  'f1',
  'f1',
  'f3',
  'f3',
  'f3',
  'f3',
  'f1',
  'f1',
  'v1',
  'f1',
  'f1',
  'f1',
  'f1',
  'f1',
  'f1',
  'f3',
  'f1',
  'f2',
  'f1',
  'f3',
  'f3',
  'f1',
  'f2',
  'f1',
  'f1',
  'v3',
  'f1',


In [60]:
all_dict_win = federer_serve_win_response.copy()
all_dict_lost = federer_serve_lost_response.copy()
for k, v in nadal_serve_win_response.items():
    if k in all_dict_win:
        all_dict_win[k].extend(v)
    else:
        all_dict_win[k] = v
        
for k, v in nadal_serve_lost_response.items():
    if k in all_dict_lost:
        all_dict_lost[k].extend(v)
    else:
        all_dict_lost[k] = v

In [61]:
from collections import Counter

In [62]:
cnt_win = Counter([i.split(' ')[0] for i in server_federer_won.Shots.values])
cnt_lost = Counter([i.split(' ')[0] for i in server_federer_lost.Shots.values])

In [63]:
serves_win = [i.split(' ')[0] for i in server_federer_won.Shots.values if cnt_win[i] > 10]
serves_lost = [i.split(' ')[0] for i in server_federer_lost.Shots.values if cnt_lost[i] > 10]

In [64]:
all_dict_win['X'] = serves_win
all_dict_lost['X'] = serves_lost

# Prompts and shots

We provide here the list of possible prompts and allowed shots – historically the most common shots – that you have to use. There are many more shots in the dataframe, but we only stick to the 100 shots provided and the 40 shots provided. It means that the historical RN/tester, will only shoot with one of the 100 prompts and you can reply using only one of the 40 shots.

In [65]:
# Loading the prompts and the shots

all_shots = pickle.load(open("allowed_shots.p", "rb"))
poss_prompts = pickle.load(open("possible_prompts.p" , "rb"))

print('Allowed shots: ', all_shots, '\n')
print('Possible prompts: ', poss_prompts)

Allowed shots:  ['4', '4+', '5', '6', '6+', 'X', 'b', 'b1', 'b17', 'b18', 'b19', 'b2', 'b27', 'b28', 'b29', 'b3', 'b37', 'b38', 'f', 'f+1', 'f+3', 'f1', 'f18', 'f2', 'f27', 'f28', 'f29', 'f3', 'f37', 'f38', 'r2', 's', 's1', 's2', 's28', 's3', 'v1', 'v3', 'z1', 'z3'] 

Possible prompts:  ['f3', 'f1', 'b2', 'b1', 'X', 'f2', 'b3', '4', '6', '5', '*', 'b28', 'f28', 'b27', 's2', 'f+3', 's3', 'n@', 'f+1', 'b18', 'd@', 'f38', 'b29', 's1', 'b38', 'f37', '#', 'w@', 'f27', 'f29', 'n#', 'd#', 'f', 'b', 's28', 'b37', 'b17', 'f1*', 'b19', 'b39', 'w#', 'r2', 'f18', 'f39', 'z3', 'v1', 'v3', 'f#', 's27', '4+', '6+', 'z1', 'b1*', 'f3*', 'm2', 'f17', 'r3', 's', 'r1', 's29', 's38', 'b+1', 'z2', 'o3', 'c6', 'v2', 'c4', 'b1w@', 'b#', 'b1n@', 'b+3', 'f19', 'b3*', 'f+2', 'f1w@', 'b3w@', 's37', 's39', 'f3n@', 'f3w@', 'm3', 'f1n@', 'b1d@', 'b2n#', 'f2d@', 'f1n#', 'b1w#', 's#', 'b1n#', 'x@', 'f1w#', 'y3', 'f2d#', 'b2d@', 'f2n#', 'f3d@', 'y1', 'b3w#', 'o1', 'f-3']


In [66]:
all_chars = set([])
for i in all_shots:
    all_chars = all_chars.union(set(i))

In [67]:
all_chars = ''.join(list(all_chars))
all_chars

'Xb872r4vs1+96z3f5'

In [68]:
mapper_win = set()
for k, v in all_dict_win.items():
    v_cnt = Counter(v)
    mapper_win = mapper_win.union(set([i for i in v_cnt.keys() if i not in all_shots]))
    
mapper_lost = set()
for k, v in all_dict_lost.items():
    v_cnt = Counter(v)
    mapper_lost = mapper_lost.union(set([i for i in v_cnt.keys() if i not in all_shots]))

In [69]:
def transform(s):
    if s.startswith('f'):
        return ''.join([i for i in s if i in all_chars])
    else:
        return ''.join([i for i in s if i in [j for j in all_chars if j!='+']])

In [70]:
mapper2_win = {i: transform(i) if transform(i) not in ['0', '4', '5', '6'] else '' for i in mapper_win}
mapper2_lost = {i: transform(i) if transform(i) not in ['0', '4', '5', '6'] else '' for i in mapper_lost}

In [71]:
mapper2_win

{'z=3': 'z3',
 'y-1': '1',
 'f3+3': 'f3+3',
 's37': 's37',
 'j3': '3',
 'o': '',
 'h3': '3',
 'b39': 'b39',
 'y+1': '1',
 'u-1': '1',
 's+3': 's3',
 'i3': '3',
 'v': 'v',
 's+1': 's1',
 'o1': '1',
 'u+3': '3',
 'f;3': 'f3',
 'f+29': 'f+29',
 'm': '',
 'm2': '2',
 'b+1': 'b1',
 'h2': '2',
 'p2': '2',
 'f+2': 'f+2',
 'o2': '2',
 'i2': '2',
 'r1': 'r1',
 's39': 's39',
 'b;28': 'b28',
 'k3': '3',
 'b+28': 'b28',
 'f39': 'f39',
 'c4': '',
 'h1': '1',
 'f17': 'f17',
 'r+1': 'r1',
 'f-3': 'f3',
 'y': '',
 'u1': '1',
 's-2': 's2',
 'r28': 'r28',
 'y+3': '3',
 'j=+1': '1',
 'f19': 'f19',
 'f+-1': 'f+1',
 'b;37': 'b37',
 's+19': 's19',
 'o-3': '3',
 'v2': 'v2',
 'b;27': 'b27',
 'b9': 'b9',
 'i=2': '2',
 'j+1': '1',
 'i1': '1',
 'c6': '',
 'r3': 'r3',
 'y2': '2',
 'f1-': 'f1',
 'f8': 'f8',
 's-1': 's1',
 'f;17': 'f17',
 'r29': 'r29',
 's38': 's38',
 'y3': '3',
 'b+3': 'b3',
 'j;1': '1',
 'o=2': '2',
 's+2': 's2',
 's27': 's27',
 'm1': '1',
 'p3': '3',
 'f;2': 'f2',
 'b+2': 'b2',
 'z+1': 'z1',
 'j

In [72]:
mapper2_lost

{'y-1': '1',
 'f9': 'f9',
 's37': 's37',
 'j3': '3',
 'b;38': 'b38',
 'h3': '3',
 'y+2': '2',
 'b39': 'b39',
 'y+1': '1',
 'i3': '3',
 's+3': 's3',
 'v': 'v',
 'u-1': '1',
 's+1': 's1',
 'o1': '1',
 'u+3': '3',
 'f;3': 'f3',
 'm': '',
 'l2': '2',
 'i+': '',
 'm2': '2',
 'b+1': 'b1',
 'z': 'z',
 's289': 's289',
 'h2': '2',
 'f+2': 'f+2',
 'o2': '2',
 'i2': '2',
 'r1': 'r1',
 'b;1': 'b1',
 'b+;1': 'b1',
 'f+17': 'f+17',
 's39': 's39',
 'b+19': 'b19',
 'b+28': 'b28',
 'f39': 'f39',
 'h1': '1',
 's;37': 's37',
 'f17': 'f17',
 'l3': '3',
 'q': '',
 'r+1': 'r1',
 'f-3': 'f3',
 'y': '',
 'f-;1': 'f1',
 'u1': '1',
 'f12': 'f12',
 'r28': 'r28',
 'y+3': '3',
 'f19': 'f19',
 'p1': '1',
 's+28': 's28',
 'b+7': 'b7',
 'b;27': 'b27',
 'v2': 'v2',
 'b9': 'b9',
 'r27': 'r27',
 'j+1': '1',
 'f;+1': 'f+1',
 'i1': '1',
 'r3': 'r3',
 'b+17': 'b17',
 's+29': 's29',
 'r': 'r',
 'f8': 'f8',
 's-1': 's1',
 's38': 's38',
 'r-1': 'r1',
 'y3': '3',
 'b+3': 'b3',
 'b-2': 'b2',
 'f+;3': 'f+3',
 'o=2': '2',
 's+2':

In [73]:
all_dict_win

{'b28': ['b2',
  'f3',
  'z3',
  'z3',
  'f2',
  'f3',
  'f-1',
  'f3',
  'f2',
  'f2',
  'f1',
  'f3',
  'j+2',
  'f1',
  'f1',
  'b2',
  'f2',
  'f1',
  'f+3',
  'f1',
  'f1',
  'f+1',
  'f1',
  'f1',
  'f1-',
  'f3',
  'f1',
  'f1',
  'f3',
  'u+1',
  'z3',
  'f1',
  'f1',
  'f3',
  'f+1',
  'f+1',
  'f+3',
  'f2',
  'f2',
  'f3',
  'f3',
  'v1',
  'f+1',
  'f+3',
  'f+1',
  'f1',
  'f1',
  'f+1',
  'f3',
  'z1',
  'f1',
  'f+3',
  'f1',
  'f+1',
  'f1',
  'f1',
  'f+1',
  'f3',
  'f+3',
  'f+1',
  'f1',
  'f3',
  'v3',
  'f+3',
  'f+3',
  'f+3',
  'f2',
  'f+3',
  'f1',
  'f+3',
  'b3',
  'f+1',
  'f2',
  'f1',
  'f+1',
  'f+1',
  'f+3',
  'f;3',
  'f3',
  'f+3',
  'f2',
  'f3',
  'f1',
  'f+1',
  'f+2',
  'f1',
  'f1',
  'f3',
  'f3',
  'f1',
  'f+3',
  'f+1',
  'f1',
  'f3',
  'f+3',
  'f+3',
  'f3',
  'v3',
  'f3',
  'f2'],
 's3': ['f+1',
  'f1',
  'f2',
  'f+',
  'b1',
  'b3',
  'f3',
  'b3',
  'f+1',
  'b1',
  'f3',
  'o3',
  'y+1',
  'b1',
  'z3',
  'b2',
  'b3',
  'u+1',
  '

In [74]:
all_dict_win2 = {}
all_dict_lost2 = {}
for k, v in all_dict_win.copy().items():
    v1 = [transform(i) for i in v if transform(i) in all_shots]
    all_dict_win2[k] = v1

for k, v in all_dict_lost.copy().items():
    v1 = [transform(i) for i in v if transform(i) in all_shots]
    all_dict_lost2[k] = v1

# Testing function for your model performances

See the file *aux_functions*.  The following function makes the historical RN become alive and play against you!

In [96]:
from aux_functions import *

In [97]:
Model, shots_opo_com = aux_fn(df, poss_prompts, all_shots)

In [98]:
# Building up the model function

#################### The threshold on shots rarity ###################
threshold = 1                                                        #
######################################################################

# Given current prompt p_t and shot s_t, return next prompt p_(t+1)
def model(prompt, shot):
    n_grams_ps = Model[poss_prompts.index(prompt), all_shots.index(shot)]
    if(len(n_grams_ps) > threshold):    # exclude rare reactions
        return np.random.choice(n_grams_ps)
    return 'n@'   # when (p, s) never happened, return unforced error in net

# Example: RN just served ('5') and we want to reply with a forehand ('f28'). This is what RN will reply (stochastic function)
model('5', 'f28')

'f2'

In [75]:
all_dict_win2

{'b28': ['b2',
  'f3',
  'z3',
  'z3',
  'f2',
  'f3',
  'f1',
  'f3',
  'f2',
  'f2',
  'f1',
  'f3',
  'f1',
  'f1',
  'b2',
  'f2',
  'f1',
  'f+3',
  'f1',
  'f1',
  'f+1',
  'f1',
  'f1',
  'f1',
  'f3',
  'f1',
  'f1',
  'f3',
  'z3',
  'f1',
  'f1',
  'f3',
  'f+1',
  'f+1',
  'f+3',
  'f2',
  'f2',
  'f3',
  'f3',
  'v1',
  'f+1',
  'f+3',
  'f+1',
  'f1',
  'f1',
  'f+1',
  'f3',
  'z1',
  'f1',
  'f+3',
  'f1',
  'f+1',
  'f1',
  'f1',
  'f+1',
  'f3',
  'f+3',
  'f+1',
  'f1',
  'f3',
  'v3',
  'f+3',
  'f+3',
  'f+3',
  'f2',
  'f+3',
  'f1',
  'f+3',
  'b3',
  'f+1',
  'f2',
  'f1',
  'f+1',
  'f+1',
  'f+3',
  'f3',
  'f3',
  'f+3',
  'f2',
  'f3',
  'f1',
  'f+1',
  'f1',
  'f1',
  'f3',
  'f3',
  'f1',
  'f+3',
  'f+1',
  'f1',
  'f3',
  'f+3',
  'f+3',
  'f3',
  'v3',
  'f3',
  'f2'],
 's3': ['f+1',
  'f1',
  'f2',
  'b1',
  'b3',
  'f3',
  'b3',
  'f+1',
  'b1',
  'f3',
  'b1',
  'z3',
  'b2',
  'b3',
  'f1',
  'b3',
  'v3',
  'z3',
  'b3',
  'v1',
  'b1',
  'f1',
  '

In [76]:
all_dict_lost2

{'f37': ['s1',
  's1',
  's3',
  'f3',
  'f1',
  'b1',
  's3',
  's3',
  'b1',
  's1',
  'b2',
  'b2',
  'b1',
  'b3',
  'f3',
  'b3',
  'b3',
  'b3',
  's2',
  'z1',
  'b3',
  'f2',
  'b1',
  'b3',
  'f1',
  'b1',
  's1',
  'b3',
  'b1',
  's1',
  'f+3',
  'f3',
  'b3',
  'f2',
  'f3',
  'f1',
  'b2',
  's2',
  'f3',
  'b1',
  'f+3',
  'f1',
  'b1',
  'b3',
  'f3',
  'b1',
  'f3',
  'f3',
  'b2',
  'f+1',
  'b3',
  's1',
  'b1',
  'b1',
  'f3',
  'f3',
  'f3',
  's2'],
 's38': ['f1', 'f1', 'f3', 'f1', 'b1', 'b1', 'f+1'],
 'b1': ['f1',
  'f2',
  'f2',
  'f1',
  'f1',
  'f1',
  'r2',
  'f3',
  'f1',
  'f3',
  'f2',
  'f2',
  'v3',
  'f3',
  'f1',
  'f1',
  'f1',
  'f3',
  'v3',
  'f3',
  'f1',
  'f2',
  'f3',
  'f1',
  'f1',
  'f3',
  'f3',
  'f3',
  'f3',
  'f1',
  'f1',
  'v1',
  'f1',
  'f1',
  'f1',
  'f1',
  'f1',
  'f1',
  'f3',
  'f1',
  'f2',
  'f1',
  'f3',
  'f3',
  'f1',
  'f2',
  'f1',
  'f1',
  'v3',
  'f1',
  'f2',
  'f2',
  'f2',
  'f2',
  'f1',
  'f2',
  'f2',
  'f1',
  

# Your turn to serve!

You can play against the historical RN: just type in the shots and see if you can beat him! You can try first with simple shots, like 'f3' or 'b2' or even with a volley 'v1'.

In [217]:
Counter(all_dict2['b2'])

Counter({'b1': 15,
         'b2': 35,
         'b3': 26,
         'f': 1,
         'f+1': 23,
         'f+3': 26,
         'f1': 122,
         'f2': 55,
         'f3': 63,
         's1': 4,
         's2': 1,
         's3': 3,
         'v1': 7,
         'v3': 9,
         'z1': 7,
         'z3': 7})

In [179]:
current_prompt = 'X'
while True:       ###### CAREFUL WHEN INTERRUPTING
    shot = input()
    next_prompt = model(current_prompt, shot)
    print(next_prompt)
    current_prompt = next_prompt
    if is_win(next_prompt) == 1:
        print('Point won!')
        break
    if is_win(next_prompt) == 0:
        print('Point lost!')
        break

KeyboardInterrupt: 

# Example and tester

We provide here an example of policy and a function with which you can test your model.

For testing your personal new policy, simply modify the first line of code and insert your policy dictionary. 

The final scoring will be done with this same function, with number_of_points_played >= 1000000 and different values of the threshold, to be sure that statistical fluctuations of the result are reduced.

In [105]:
# Test function that makes your policy (a.k.a. Phederos) play against historical RN !!

############################## Modify here! ##########################
example_policy = cnts2
######################################################################

number_of_points_played = 150000


serves_opponent = [shot for shot in [sh[0] for sh in shots_opo_com] if shot[0] in '456']
points_won  = 0
points_lost = 0


for num in range(number_of_points_played):
    print(num, end = '\r')
    # Random choice on who is serving
    if np.random.randint(0,2) == 0:
        current_prompt = 'X'
    else :
        current_prompt = np.random.choice(serves_opponent)
    while True:
        shot = np.random.choice(example_policy.get(current_prompt,['5']))
        next_prompt = model(current_prompt, shot)
        current_prompt = next_prompt
        if is_win(current_prompt) == 1:
            points_won  += 1
            break
        if is_win(current_prompt) == 0:
            points_lost += 1
            break
            
print('How well are we performing? '     , points_won/(points_lost + points_won))
print('Baseline historical performance: ', 1 - (df['Winner'].sum()/len(df['Winner'])))

How well are we performing?  0.62894
Baseline historical performance:  0.5030959752321982


# Your turn now!
Show us that you are up for the challenge and make your Phederos destroy the historical RN! Just be careful and do not get carried away... real challenges lie ahead of you all!

In [143]:
is_win('w*')

0

In [78]:
from collections import Counter

In [88]:
all_dict = {}
for key, value in all_dict_win2.items():
    win_len = len(value)
    if key in all_dict_lost2:
        lost_len = len(all_dict_lost2[key])
    else:
        lost_len = 0
    tot_len = win_len+lost_len
    print(tot_len)
    win_count = Counter(value)
    if lost_len > 0:
        lost_count = Counter(value)
    else:
        lost_count = {}
    for resp, count in win_count.most_common():
        if resp in lost_count:
            
    #print(val_count.most_common())

178
f1 28
f3 20
f+3 15
f+1 14
f2 10
z3 3
b2 2
v3 2
v1 1
z1 1
b3 1
102
f3 17
b3 15
f1 8
b1 7
z3 6
f+1 5
f2 3
b2 3
v3 1
v1 1
f+3 1
s1 1
109
f1 10
b3 8
f+1 7
b1 6
f3 6
s1 4
b2 4
z1 3
f2 1
z3 1
s2 1
126
f+1 21
f+3 15
f3 14
z3 7
v1 7
f1 6
b2 3
z1 3
f2 3
v3 3
b3 2
1955
b3 192
b2 191
b1 120
s1 49
s3 44
f3 42
f1 32
s2 30
f2 22
z1 13
f+1 6
z3 4
f+3 2
v1 1
b 1
1
z3 1
71
f1 14
f2 6
f3 5
b1 2
b3 1
87
f+3 10
f+1 9
f1 9
f3 8
z3 7
b3 5
v3 3
v1 2
s1 2
b2 2
s2 1
f2 1
566
f1 71
f3 42
f2 40
b2 39
b1 23
b3 14
f+1 13
v1 7
z1 6
f+3 4
s2 2
s1 2
f 1
z3 1
756
f1 122
f3 63
f2 55
b2 35
f+3 26
b3 26
f+1 23
b1 15
v3 9
z1 7
z3 7
v1 7
s1 4
s3 3
f 1
s2 1
797
f1 180
f3 74
f2 51
v3 21
f+3 15
f+1 11
v1 10
f 2
r2 2
1
f2 1
114
b1 10
f1 10
f3 7
f2 6
b2 6
b3 6
s1 5
s2 1
f+3 1
f+1 1
150
f1 25
f2 17
f3 15
f+1 11
f+3 2
b3 2
v3 1
557
b3 67
b1 52
b2 42
z3 23
s1 22
f3 13
f1 13
s3 8
f2 6
z1 5
s2 4
f+1 2
v3 1
7
f1 2
f+3 2
z3 1
z1 1
543
f1 126
f2 35
f3 30
v1 15
r2 2
f+1 2
88
f1 19
f3 7
f2 4
f+3 3
f+1 2
r2 1
5
b 1
f1 1
80
f3 12
f+1 1

In [91]:
cnts = {}
for k, v in all_dict_win2.copy().items():
    tmp_cnt = Counter(v)
    cnts[k] = {tmp_cnt}
for k, v in all_dict_lost2.copy().items():
    tmp_cnt = Counter(v)
    if k not in cnts:
        cnts[k]={i:-j for i, j in tmp_cnt.items()}
    else:
        for i, j in tmp_cnt.items():
            if i in cnts[k]:
                cnts[k][i] -= j
            else:
                cnts[k][i] = -j
    
cnts

{'b28': Counter({'b2': -1,
          'f3': 3,
          'z3': 3,
          'f2': -3,
          'f1': -2,
          'f+3': 13,
          'f+1': 7,
          'v1': 0,
          'z1': 0,
          'v3': 2,
          'b3': -2,
          'f': -1,
          'b1': -3}),
 's3': Counter({'f+1': 3,
          'f1': 0,
          'f2': 2,
          'b1': 3,
          'b3': 8,
          'f3': 13,
          'z3': 4,
          'b2': 0,
          'v3': 1,
          'v1': 1,
          'f+3': 1,
          's1': 1,
          'z1': -2,
          's3': -1}),
 'f37': Counter({'s1': -2,
          'z1': 2,
          'b1': -5,
          'f2': -1,
          'f3': -5,
          'b3': -2,
          'f1': 6,
          'f+1': 6,
          'b2': 0,
          'z3': 1,
          's2': -2,
          's3': -3,
          'f+3': -2}),
 'b27': Counter({'f+3': 6,
          'b2': 2,
          'z1': 2,
          'f3': 8,
          'z3': 6,
          'f2': 1,
          'f+1': 12,
          'f1': 2,
          'v3': -1,
         

In [102]:
cnts2 = {}

for k, v in cnts.items():
    tmp = []
    for i, j in v.items():
        if j < 1:
            continue
        else:
            tmp.extend([i for _ in range(j)])
    if len(tmp) == 0:
        continue
    cnts2[k] = tmp
cnts2

{'b28': ['f3',
  'f3',
  'f3',
  'z3',
  'z3',
  'z3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'v3',
  'v3'],
 's3': ['f+1',
  'f+1',
  'f+1',
  'f2',
  'f2',
  'b1',
  'b1',
  'b1',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'f3',
  'f3',
  'f3',
  'f3',
  'f3',
  'f3',
  'f3',
  'f3',
  'f3',
  'f3',
  'f3',
  'f3',
  'f3',
  'z3',
  'z3',
  'z3',
  'z3',
  'v3',
  'v1',
  'f+3',
  's1'],
 'f37': ['z1',
  'z1',
  'f1',
  'f1',
  'f1',
  'f1',
  'f1',
  'f1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'z3'],
 'b27': ['f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'b2',
  'b2',
  'z1',
  'z1',
  'f3',
  'f3',
  'f3',
  'f3',
  'f3',
  'f3',
  'f3',
  'f3',
  'z3',
  'z3',
  'z3',
  'z3',
  'z3',
  'z3',
  'f2',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f

In [89]:
all_dict_lost2

{'f37': ['s1',
  's1',
  's3',
  'f3',
  'f1',
  'b1',
  's3',
  's3',
  'b1',
  's1',
  'b2',
  'b2',
  'b1',
  'b3',
  'f3',
  'b3',
  'b3',
  'b3',
  's2',
  'z1',
  'b3',
  'f2',
  'b1',
  'b3',
  'f1',
  'b1',
  's1',
  'b3',
  'b1',
  's1',
  'f+3',
  'f3',
  'b3',
  'f2',
  'f3',
  'f1',
  'b2',
  's2',
  'f3',
  'b1',
  'f+3',
  'f1',
  'b1',
  'b3',
  'f3',
  'b1',
  'f3',
  'f3',
  'b2',
  'f+1',
  'b3',
  's1',
  'b1',
  'b1',
  'f3',
  'f3',
  'f3',
  's2'],
 's38': ['f1', 'f1', 'f3', 'f1', 'b1', 'b1', 'f+1'],
 'b1': ['f1',
  'f2',
  'f2',
  'f1',
  'f1',
  'f1',
  'r2',
  'f3',
  'f1',
  'f3',
  'f2',
  'f2',
  'v3',
  'f3',
  'f1',
  'f1',
  'f1',
  'f3',
  'v3',
  'f3',
  'f1',
  'f2',
  'f3',
  'f1',
  'f1',
  'f3',
  'f3',
  'f3',
  'f3',
  'f1',
  'f1',
  'v1',
  'f1',
  'f1',
  'f1',
  'f1',
  'f1',
  'f1',
  'f3',
  'f1',
  'f2',
  'f1',
  'f3',
  'f3',
  'f1',
  'f2',
  'f1',
  'f1',
  'v3',
  'f1',
  'f2',
  'f2',
  'f2',
  'f2',
  'f1',
  'f2',
  'f2',
  'f1',
  

In [94]:
cnts2['X']

['4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6'

In [100]:
current_prompt

'f39'

In [101]:
cnts2['f39']

[]